In [1]:
import splat
import splat.plot as splot
import splat.photometry as sphot
import splat.empirical as spem

# other useful imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import astropy.units as u
from astropy.io import fits
from astropy.utils.data import download_file

# Python standard library
import time
import warnings
import math
from itertools import combinations


# Third-party software
import numpy as np
import numpy.ma as ma

# Astropy
from astropy import coordinates as coord
from astropy import units as u
from astropy.table import Table, vstack, hstack, join
from astropy.coordinates import SkyCoord
from astropy.io import fits

# Astroquery. This tutorial requires 0.3.5 or greater.
import astroquery
from astroquery.simbad import Simbad
from astroquery.vo_conesearch import conf, conesearch, vos_catalog
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('/Users/cdann/Downloads/splat-master/splat-master/resources/Spectra/SPEX-PRISM/')

Adding 145 sources from C:\Users\cdann\Downloads\splat-master\splat-master/resources/Data/Public/LRIS-RED/ to spectral database
gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!
Could not import regions, which is required for some of the functionalities of this module.


In [2]:
cols = {'2MASS':( '_r', 'RAJ2000', 'DEJ2000', 'errMaj', 'errMin', 'errPA', '_2MASS', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 
                     'Kmag', 'e_Kmag', 'Date', '_tab1_36'), 
               'SDSS':('_r', 'RA_ICRS', 'e_RA_ICRS', 'DE_ICRS', 'e_DE_ICRS', 'class', 'SDSS12', 'objID', 'SpObjID', 'umag',
                     'e_umag', 'ObsDate', 'gmag', 'e_gmag', 'rmag', 'e_rmag', 'imag', 'e_imag', 'zmag', 'e_zmag', 'spType',
                     'spCl', 'subCl', 'spS_N', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE'),
               'DES':('_r', 'RAJ2000', 'DEJ2000', 'DES', 'S_Gg', 'S_Gr', 'S_Gi', 'S_Gz', 'S_GY', 'gmag', 'rmag', 'imag', 'zmag',
                     'Ymag', 'e_gmag', 'e_rmag', 'e_imag', 'e_zmag', 'e_Ymag', 'EBV'),
               'VISTA':('_r', 'RAJ2000', 'DEJ2000', 'SrcID', 'Name', 'Mclass', 'E_B-V_', 'AY', 'AJ', 'AH', 'AKs', '_tab1_36', 'Yap3',
                     'e_Yap3', '_tab1_58', 'Jap3', 'e_Jap3', '_tab1_82', 'Hap3', 'e_Hap3', '_tab1_106', 'Ksap3', 'e_Ksap3'), 
               'ALLWISE':('_r', 'RAJ2000', 'DEJ2000', 'AllWISE', 'eeMaj', 'eeMin', 'eePA', 'W1mag', 'e_W1mag', 'W2mag', 'e_W2mag',
                     'W3mag', 'e_W3mag', 'W4mag', 'e_W4mag', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag', 'ex', 'pmRA', 
                     'e_pmRA', 'pmDE', 'e_pmDE'),
               'PANSTARRS':('_r', 'RAJ2000', 'DEJ2000', 'objID', 'e_RAJ2000', 'e_DEJ2000', '_tab1_10', 'gmag', 'e_gmag','rmag',
                     'e_rmag', 'imag', 'e_imag', 'zmag', 'e_zmag','ymag', 'e_ymag'), 
               'GAIA':('_r', 'DR2Name', 'RA_ICRS', 'e_RA_ICRS', 'DE_ICRS', 'e_DE_ICRS', 'Source', 'Epoch', 'Plx', 'e_Plx', 
                     'pmRA', 'e_pmRA','pmDE', 'e_pmDE', 'Gmag', 'e_Gmag', 'BPmag', 'e_BPmag', 'RPmag', 'e_RPmag', 'RV',
                     'e_RV', 'Teff', 'AG', 'E_BP-RP_', 'Rad', 'Lum'),
               'UKIDSS':('_r', 'RAJ2000', 'e_RAJ2000', 'DEJ2000', 'e_DEJ2000', 'ULAS', 'Ymag', 'e_Ymag', 'Jmag1', 'e_Jmag1',
                     'Jmag2', 'e_Jmag2', 'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag', 'ID', 'Epoch','pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'cl'),
               'DENIS':('_r', 'RAJ2000', 'DEJ2000', 'Imag', 'e_Imag', 'Jmag', 'e_Jmag', 'Kmag', 'e_Kmag', 'Rmag', 'Bmag', 'ObsJD',
                     'DENIS')}
catalogs = {'2MASS', 'SDSS', 'DES', 'VISTA', 'ALLWISE', 'PANSTARRS', 'GAIA', 'UKIDSS', 'DENIS'}
v2 = Vizier(columns=['+_r', '**'])
catalog_files = {'2MASS':'II/246/out', 'SDSS':'V/147/sdss12', 'DES':'II/357/des_dr1', 'VISTA':'II/359/vhs_dr4', 
                'ALLWISE':'II/328/allwise', 'PANSTARRS':'II/349/ps1', 'GAIA':'I/345/gaia2', 'UKIDSS':'II/319/las9',
                'DENIS':'B/denis/denis'}
col_rename = {'2MASS': ('2MASS_r', '2MASS_RA', '2MASS_DEC', '2MASS_POS_E_MAJ', '2MASS_POS_E_MIN', '2MASS_POS_E_PA', '2MASS_DESIGNATION',
                       '2MASS_JMAG', '2MASS_JMAG_E', '2MASS_HMAG', '2MASS_HMAG_E', '2MASS_KMAG', '2MASS_KMAG_E', '2MASS_EPOCH_YEAR', 
                       '2MASS_EPOCH_JD'),
             'SDSS':('SDSS_r', 'SDSS_RA', 'SDSS_RA_E', 'SDSS_DEC', 'SDSS_DEC_E', 'SDSS_TYPE', 'SDSS_DESIGNATION', 'SDSS_ID',  
                      'SDSS_SPECTRUM_ID','SDSS_uMAG', 'SDSS_uMAG_E', 'SDSS_EPOCH_YEAR', 'SDSS_gMAG', 'SDSS_gMAG_E',
                     'SDSS_rMAG', 'SDSS_rMAG_E', 'SDSS_iMAG', 'SDSS_iMAG_E', 'SDSS_zMAG', 'SDSS_zMAG_E', 'SDSS_SPECTRUM_SPT',
                     'SDSS_SPECTRUM_CLASS', 'SDSS_SPECTRUM_TYPE', 'SDSS_SPECTRUM_SNR', 'SDSS_PMRA', 'SDSS_PMRA_E', 'SDSS_PMDEC', 
                     'SDSS_PMDEC_E'),
             'DES':('DES_r', 'DES_RA', 'DES_DEC', 'DES_DESIGNATION', 'DES_EXTENDED_FLAG_g', 'DES_EXTENDED_FLAG_r', 'DES_EXTENDED_FLAG_i',
                    'DES_EXTENDED_FLAG_z', 'DES_EXTENDED_FLAG_Y', 'DES_gMAG', 'DES_rMAG', 'DES_iMAG', 'DES_zMAG', 'DES_YMAG', 'DES_gMAG_E',
                    'DES_rMAG_E', 'DES_iMAG_E', 'DES_zMAG_E', 'DES_YMAG_E', 'DES_EBV'),
             'VISTA':('VISTA_r', 'VISTA_RA', 'VISTA_DEC', 'VISTA_ID', 'VISTA_DESIGNATION', 'VISTA_TYPE', 'VISTA_EBV', 'VISTA_AY',
                      'VISTA_AJ', 'VISTA_AH', 'VISTA_AK', 'VISTA_YMAG_EPOCH_MJD', 'VISTA_YMAG', 'VISTA_YMAG_E', 'VISTA_JMAG_EPOCH_MJD',
                      'VISTA_JMAG', 'VISTA_JMAG_E', 'VISTA_HMAG_EPOCH_MJD', 'VISTA_HMAG', 'VISTA_HMAG_E', 'VISTA_KMAG_EPOCH_MJD', 'VISTA_KMAG',
                      'VISTA_KMAG_E'),
             'ALLWISE':('ALLWISE_r', 'ALLWISE_RA', 'ALLWISE_DEC', 'ALLWISE_DESIGNATION', 'ALLWISE_POS_E_MAJ', 'ALLWISE_POS_E_MIN', 
                        'ALLWISE_POS_E_PA', 'ALLWISE_W1MAG', 'ALLWISE_W1MAG_E', 'ALLWISE_W2MAG', 'ALLWISE_W2MAG_E', 'ALLWISE_W3MAG', 
                        'ALLWISE_W3MAG_E', 'ALLWISE_W4MAG', 'ALLWISE_W4MAG_E', 'ALLWISE_JMAG', 'ALLWISE_JMAG_E', 'ALLWISE_HMAG', 
                        'ALLWISE_HMAG_E', 'ALLWISE_KMAG', 'ALLWISE_KMAG_E', 'ALLWISE_EXTENDED_FLAG', 'ALLWISE_PMRA', 'ALLWISE_PMRA_E',
                        'ALLWISE_PMDEC', 'ALLWISE_PMDEC_E'),
             'PANSTARRS':('PANSTARRS_r', 'PANSTARRS_RA', 'PANSTARRS_DEC', 'PANSTARRS_DESIGNATION', 'PANSTARRS_RA_E', 'PANSTARRS_DEC_E', 
                          'PANSTARRS_EPOCH_MJD', 'PANSTARRS_gMAG', 'PANSTARRS_gMAG_E', 'PANSTARRS_rMAG', 'PANSTARRS_rMAG_E', 'PANSTARRS_iMAG',
                          'PANSTARRS_iMAG_E', 'PANSTARRS_zMAG', 'PANSTARRS_zMAG_E', 'PANSTARRS_yMAG', 'PANSTARRS_yMAG_E'),
             'GAIA':('GAIA_r', 'GAIA_DESIGNATION', 'GAIA_RA', 'GAIA_RA_E', 'GAIA_DEC', 'GAIA_DEC_E', 'GAIA_ID', 'GAIA_EPOCH_YEAR', 
                     'GAIA_PLX', 'GAIA_PLX_E', 'GAIA_PMRA', 'GAIA_PMRA_E', 'GAIA_PMDEC', 'GAIA_PMDEC_E', 'GAIA_GMAG', 'GAIA_GMAG_E',
                     'GAIA_BPMAG', 'GAIA_BPMAG_E', 'GAIA_RPMAG', 'GAIA_RPMAG_E', 'GAIA_RV', 'GAIA_RV_E', 'GAIA_TEFF', 'GAIA_AG', 'GAIA_EBR',
                     'GAIA_RADIUS', 'GAIA_LUMINOSITY'),
             'UKIDSS':('UKIDSS_r', 'UKIDSS_RA', 'UKIDSS_RA_E', 'UKIDSS_DEC', 'UKIDSS_DEC_E', 'UKIDSS_DESIGNATION', 'UKIDSS_YMAG', 'UKIDSS_YMAG_E',
                       'UKIDSS_EPOCH1_JMAG', 'UKIDSS_EPOCH1_JMAG_E', 'UKIDSS_EPOCH2_JMAG', 'UKIDSS_EPOCH2_JMAG_E', 'UKIDSS_HMAG', 'UKIDSS_HMAG_E',
                       'UKIDSS_KMAG', 'UKIDSS_KMAG_E', 'UKIDSS_ID', 'UKIDSS_EPOCH_YEAR', 'UKIDSS_PMRA', 'UKIDSS_PMRA_E', 'UKIDSS_PMDEC', 
                       'UKIDSS_PMDEC_E', 'UKIDSS_TYPE'),
              'DENIS':('DENIS_r', 'DENIS_RA', 'DENIS_DEC', 'DENIS_IMAG', 'DENIS_IMAG_E', 'DENIS_JMAG', 'DENIS_JMAG_E', 'DENIS_KMAG', 
                      'DENIS_KMAG_E', 'DENIS_RMAG', 'DENIS_BMAG', 'DENIS_EPOCH_JD', 'DENIS_DESIGNATION')}

In [3]:
class Source:
    #initial function will take the information required to get the coordinates from the fits file
    
    
    def __init__(self, path_name):
        Header = fits.open(path_name)[0].header
        for x in range(len(list(Header))):
            if 'RA' == list(Header)[x] or '_RA' in list(Header)[x]:
                RA_header = list(Header)[x]
            if 'DEC' == list(Header)[x] or '_DEC' in list(Header)[x]:
                DEC_header = list(Header)[x]
        
        try:
            self.RA = Header[RA_header]
            self.DEC = Header[DEC_header]
        except UnboundLocalError as error:
            #This means that RA_header or DEC_header did not get a value bound to them.  Could either mean that the Header does
            #not contain an RA or DEC value, or that the RA/DEC value is not recognized
            print(list(Header))
            print("The function could not find a RA or DEC label in the above Header.  Double check to make sure that the \
                   given file contains the coordinates")

        
    def set_unit(self):
        if ':' in self.RA:
            self.RA = convert_coords(self.RA, self.DEC)[0]
            self.DEC = convert_coords(self.RA, self.DEC)[1]
            self.unit = 'degree'
        else:
            self.unit = 'degree'
            
    def cat_num(self):
        self.given_catalogs = []
        for x in range(len(catalogs)):
            if 'No data' in cat_results(self.RA, self.DEC, cat = catalogs[x], unit = self.unit):
                continue
            else:
                self.given_catalogs.append(catalogs[x])

In [4]:
def cat_results(RA, DEC, cat, unit = 'ha', rad = '30s'):
    count = list(catalogs).count(cat)
    if count > 0:
        #print("Proper Catalog")
        if unit is 'ha':
            catalog_result = v2.query_region(coord.SkyCoord(ra=RA, dec=DEC,
                                                            unit=(u.hourangle, u.degree),
                                                            frame='icrs'),
                                            radius= rad,
                                            catalog=[cat])
                        
#            return(catalog_result[catalog_files[cat]])
        if unit is 'degree':
            catalog_result = v2.query_region(coord.SkyCoord(ra=RA, dec=DEC,
                                                            unit=(u.degree, u.degree),
                                                            frame='icrs'),
                                            radius = rad,
                                            catalog=[cat])
#            return(catalog_result[catalog_files[cat]])        
    else:
        return("Improper Catalog")
        
    if any(catalog_result) == True:
        revise = catalog_result[catalog_files[cat]]
        temp_table = Table([])
        subs = list(cols[cat])
        
        for x in range(len(subs)):
            count_cols = revise.colnames.count(subs[x])
            if count_cols > 0:
                temp_table = hstack([temp_table, revise[subs[x]]])
        for y in range(len(list(temp_table.colnames))):
            temp_table.rename_column(cols[cat][y], col_rename[cat][y])
        return(temp_table)
    else:
        return("No data")

In [5]:
def comb_cat(RA, DEC, given_cat, unit = 'degree', rad = '30s'):
    #Function starts by searching the first given catalog (usually 2MASS)
    initial_results = Table(cat_results(RA, DEC, given_cat[0], unit, rad))
    possible_sources = len(list(initial_results))
    comp_table = Table([])
    #For each source returned by 2MASS, record the source coordinates and run those coordinates through the list of catalogs given.
    #Record the closest matches that each catalog returns.

    for x in range(possible_sources):
        new_RA = initial_results[x][given_cat[0]+'_RA']
        new_DEC = initial_results[x][given_cat[0]+'_DEC']
        temp_table = Table([])
        for y in range(len(given_cat)):
            temp_table = hstack([temp_table, cat_results(new_RA, new_DEC, given_cat[y], unit = 'degree', rad = rad)[0]])
        comp_table = vstack([comp_table, temp_table])
                            

    return(comp_table)

In [6]:
#Need to find a way to take value such as 2015.5 and turn it into a year, month, and day or straight into Julian Date

def convert_year(value):
    leap_years = [1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
    year = math.floor(value)
    str_value = str(value)
    month_day = str_value[4:]
    if year in leap_years:
        day_number = float(month_day) * 366
        month_value = math.floor(day_number)
        if month_value in range(0,31):
            month = 1
            day = month_value
        if month_value in range(31,60):
            month = 2
            day = month_value - 31
        if month_value in range(60,91):
            month = 3
            day = month_value - 60
        if month_value in range(91,121):
            month = 4
            day = month_value - 91
        if month_value in range(121,152):
            month = 5
            day = month_value - 121
        if month_value in range(152,182):
            month = 6
            day = month_value - 152
        if month_value in range(182,213):
            month = 7
            day = month_value - 182
        if month_value in range(213,244):
            month = 8
            day = month_value - 213
        if month_value in range(244,274):
            month = 9
            day = month_value - 244
        if month_value in range(274,305):
            month = 10
            day = month_value - 274
        if month_value in range(305,335):
            month = 11
            day = month_value - 305
        if month_value in range(335,366):
            month = 12
            day = month_value - 335
    else:
        day_number = float(month_day) * 365
        month_value = math.floor(day_number)
        if month_value in range(0,31):
            month = 1
            day = month_value
        if month_value in range(31,59):
            month = 2
            day = month_value - 31
        if month_value in range(59,90):
            month = 3
            day = month_value - 59
        if month_value in range(90,120):
            month = 4
            day = month_value - 90
        if month_value in range(120,151):
            month = 5
            day = month_value - 120
        if month_value in range(151,181):
            month = 6
            day = month_value - 151
        if month_value in range(181,212):
            month = 7
            day = month_value - 181
        if month_value in range(212,243):
            month = 8
            day = month_value - 212
        if month_value in range(243,273):
            month = 9
            day = month_value - 243
        if month_value in range(273,304):
            month = 10
            day = month_value - 273
        if month_value in range(304,334):
            month = 11
            day = month_value - 304
        if month_value in range(334,365):
            month = 12
            day = month_value - 334
            
    return(year, month, day)

In [7]:
def date_to_jd(year,month,day):
    """
    Convert a date to Julian Day.
    
    Algorithm from 'Practical Astronomy with your Calculator or Spreadsheet', 
        4th ed., Duffet-Smith and Zwart, 2011.
    
    Parameters
    ----------
    year : int
        Year as integer. Years preceding 1 A.D. should be 0 or negative.
        The year before 1 A.D. is 0, 10 B.C. is year -9.
        
    month : int
        Month as integer, Jan = 1, Feb. = 2, etc.
    
    day : float
        Day, may contain fractional part.
    
    Returns
    -------
    jd : float
        Julian Day
        
    Examples
    --------
    Convert 6 a.m., February 17, 1985 to Julian Day
    
    >>> date_to_jd(1985,2,17.25)
    2446113.75
    
    """
    if month == 1 or month == 2:
        yearp = year - 1
        monthp = month + 12
    else:
        yearp = year
        monthp = month
    
    # this checks where we are in relation to October 15, 1582, the beginning
    # of the Gregorian calendar.
    if ((year < 1582) or
        (year == 1582 and month < 10) or
        (year == 1582 and month == 10 and day < 15)):
        # before start of Gregorian calendar
        B = 0
    else:
        # after start of Gregorian calendar
        A = math.trunc(yearp / 100.)
        B = 2 - A + math.trunc(A / 4.)
        
    if yearp < 0:
        C = math.trunc((365.25 * yearp) - 0.75)
    else:
        C = math.trunc(365.25 * yearp)
        
    D = math.trunc(30.6001 * (monthp + 1))
    
    jd = B + C + D + day + 1720994.5
    
    return jd

In [9]:
def confirm_source(RA, DEC):
    #First take the original RA and DEC and run a 2MASS search for those coordinates within 30 arcseconds
    initial_results = Table(MASS_results_deg(RA, DEC))
    possible_sources = len(list(initial_results))
    comp_table = Table([])
#    comp_table = Table([])
    
    #For each source returned by 2MASS, record the source coordinates and run those coordinates through the list of catalogs given.
    #Record the closest matches that each catalog returns.

    for x in range(possible_sources):
        MASS_results = initial_results[x]
        new_RA = MASS_results['RA.']
        new_DEC = MASS_results['DEC.']
        new_coords = (new_RA, new_DEC)
        temp_table = compare_catalogs(new_RA, new_DEC, '2MASS')
        comp_table = vstack([comp_table, temp_table]) 
    
#    return(comp_table)
    #Using the magnitudes given from each source and taking into account their uncertainty, create a color-index to determine
    # the color of each source.  If the source is not the correct color, flag it.
#    MASS_mag_table = Table([comp_table['2MASS-Jmag'], comp_table['2MASS-e_Jmag'], comp_table['2MASS-Hmag'], comp_table['2MASS-e_Hmag'],
#                           comp_table['2MASS-Kmag'], comp_table['2MASS-e_Kmag']])
    
    #Check if any of the values are masked
#    for z in range(len(list(MASS_mag_table))):
#        if MASS_mag_table['2MASS-Jmag'][z] is ma.masked:
#            MASS_mag_table['2MASS-Jmag'][z] = 0
#        if MASS_mag_table['2MASS-e_Jmag'][z] is ma.masked:
#            MASS_mag_table['2MASS-e_Jmag'][z] = 0
#        if MASS_mag_table['2MASS-Hmag'][z] is ma.masked:
#            MASS_mag_table['2MASS-Hmag'][z] = 0
#        if MASS_mag_table['2MASS-e_Hmag'][z] is ma.masked:
#            MASS_mag_table['2MASS-e_Hmag'][z] = 0
#        if MASS_mag_table['2MASS-Kmag'][z] is ma.masked:
#            MASS_mag_table['2MASS-Kmag'][z] = 0
#        if MASS_mag_table['2MASS-e_Kmag'][z] is ma.masked:
#            MASS_mag_table['2MASS-e_Kmag'][z] = 0
#        
#    MASS_err_up = Table([MASS_mag_table['2MASS-Jmag'] + MASS_mag_table['2MASS-e_Jmag'], MASS_mag_table['2MASS-Hmag'] +
#                        MASS_mag_table['2MASS-e_Hmag'], MASS_mag_table['2MASS-Kmag'] + MASS_mag_table['2MASS-e_Kmag']])
#    MASS_err_up['2MASS-Jmag'].name = '2MASS_Jmag_up'
#    MASS_err_up['2MASS-Hmag'].name = '2MASS_Hmag_up'
#    MASS_err_up['2MASS-Kmag'].name = '2MASS_Kmag_up'
#    MASS_err_down = Table([MASS_mag_table['2MASS-Jmag'] - MASS_mag_table['2MASS-e_Jmag'], MASS_mag_table['2MASS-Hmag'] -
#                           MASS_mag_table['2MASS-e_Hmag'], MASS_mag_table['2MASS-Kmag'] - MASS_mag_table['2MASS-e_Kmag']])
#    MASS_err_down['2MASS-Jmag'].name = '2MASS_Jmag_down'
#    MASS_err_down['2MASS-Hmag'].name = '2MASS_Hmag_down'
#    MASS_err_down['2MASS-Kmag'].name = '2MASS_Kmag_down'
#    MASS_col_index_up = Table([MASS_err_up['2MASS_Jmag_up'] - MASS_err_up['2MASS_Hmag_up'], 
#                               MASS_err_up['2MASS_Hmag_up'] - MASS_err_up['2MASS_Kmag_up']])
#    MASS_col_index_up['2MASS-Jmag'].name = '2MASS_J-H_up'
#    MASS_col_index_up['2MASS-Hmag'].name = '2MASS_H-K_up'
#    MASS_col_index_down = Table([MASS_err_down['2MASS_Jmag_down'] - MASS_err_down['2MASS_Hmag_down'], 
#                               MASS_err_down['2MASS_Hmag_down'] - MASS_err_down['2MASS_Kmag_down']])
#    MASS_col_index_down['2MASS-Jmag'].name = '2MASS_J-H_down'
#    MASS_col_index_down['2MASS-Hmag'].name = '2MASS_H-K_down'
#    
#    MASS_col_index = Table([(MASS_col_index_up['2MASS_J-H_up'] + MASS_col_index_down['2MASS_J-H_down'])*0.5,
#                           (MASS_col_index_up['2MASS_H-K_up'] + MASS_col_index_down['2MASS_H-K_down'])*0.5])
#    MASS_col_index['2MASS-Jmag'].name = '2MASS_J-H'
#    MASS_col_index['2MASS-Hmag'].name = '2MASS_H-K'
#    MASS_col_index['2MASS_J-H_err'] = MASS_col_index_up['2MASS_J-H_up'] - MASS_col_index['2MASS_J-H']
#    MASS_col_index['2MASS_H-K_err'] = MASS_col_index_up['2MASS_H-K_up'] - MASS_col_index['2MASS_H-K']
#    
#    #return(MASS_col_index)
#    comp_table['col_flag'] = 'NO'
#    for z in range(len(list(MASS_col_index))):
#        if (MASS_col_index['2MASS_J-H'][z] < 0.56) or (MASS_col_index['2MASS_H-K'][z] < 0.18):
#            if (MASS_col_index['2MASS_J-H'][z] + MASS_col_index['2MASS_J-H_err'][z] < 0.56) or (MASS_col_index['2MASS_H-K'][z]
#            + MASS_col_index['2MASS_H-K_err'][z] < 0.18):
#                comp_table['col_flag'][z] = 'YES'
#            else:
#                continue
#        else:
#            continue
        
    #Since this function could end up returning 9 different measurements per source, the function needs to first determine which 
    #sources will be returned from the catalogs.  If only two measurements per source are returned, the proper motion can be 
    #calculated based on a line.  If more than 2 measurements per source are returned, then the proper motions of both RA and DEC
    # must be calculated using a line fit graph with chi squared statistic to check the fit.
    info_table = Table([])
    disregard = ['DES', 'ALLWISE']
    for x in range(len(list(comp_table.colnames))):
        temp_table = Table([])
        if "_RA" in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "_DEC" in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "_MAJ" in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "_MIN" in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "_PA" in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "pmRA." in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "pmDEC." in comp_table.colnames[x]:
            if disregard[0] in comp_table.colnames[x]:
                continue
            if disregard[1] in comp_table.colnames[x]:
                continue
            else:
                temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        if "Epoch_" in comp_table.colnames[x]:
            if "2MASS_Epoch_Year" in comp_table.colnames[x]:
                continue
            if "VISTA_Ymag_Epoch_MJD" in comp_table.colnames[x]:
                continue
            if "VISTA_Hmag_Epoch_MJD" in comp_table.colnames[x]:
                continue
            if "VISTA_Kmag_Epoch_MJD" in comp_table.colnames[x]:
                continue
            else:
                if "Epoch_Year" in comp_table.colnames[x]:
                    #Need to convert from year to Julian Date
                    temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
                    for z in range(len(list(comp_table[comp_table.colnames[x]]))):
                        bad_value = comp_table[comp_table.colnames[x]][z]
                        con_value = convert_year(bad_value)
                        JD_value = date_to_jd(con_value[0], con_value[1], con_value[2])
                        temp_table[comp_table.colnames[x]][z] = JD_value
                    temp_table[comp_table.colnames[x]].unit = u.day
                    temp_table[comp_table.colnames[x]].name = comp_table.colnames[x] + "_to_JD"
                else:
                    temp_table[comp_table.colnames[x]] = comp_table[comp_table.colnames[x]]
        info_table = hstack([info_table, temp_table])
    
    for x in range(len(list(info_table.colnames))):
        if "_MJD" in info_table.colnames[x]:
            info_table[info_table.colnames[x]] =  2400000.5 + info_table[info_table.colnames[x]]
            info_table[info_table.colnames[x]].name = info_table.colnames[x] + "_to_JD"    
    
    #With all of the necessary information, first create subplots of RA and DEC proper motion for each row
    pm_table = Table([])
    for z in range(len(list(info_table[info_table.colnames[0]]))):
        RA_values = []
        time_values = []
        DEC_values = []
        
        for x in range(len(list(info_table.colnames))):     
            if "_RA." in info_table.colnames[x]:
                RA_values.append(info_table[info_table.colnames[x]][z])
            if "Epoch" in info_table.colnames[x]:
                time_values.append(info_table[info_table.colnames[x]][z])
            if "_DEC." in info_table.colnames[x]:
                DEC_values.append(info_table[info_table.colnames[x]][z])
        #Now that we have all the values for each row in a table, create a function that determines how many measurements are
        #being used and act accordingly
        measure_num = len(RA_values)  
        if measure_num < 2:
            return("Not enough measurements")
        if measure_num == 2:
            #For 2 or less measurements, a simple plot is formed with nothing extra added
            fig = plt.figure(figsize=(15, 5))
            ax1 = fig.add_subplot(1,2,1)
            ax2 = fig.add_subplot(1,2,2)
            ax1.plot(time_values, RA_values, '-ok')
            ax2.plot(time_values, DEC_values, '-ok')
            ax1.set_ylabel('RA coords (deg)')
            ax1.set_xlabel('Time (days)')
            ax2.set_ylabel('DEC coords (deg)')
            ax2.set_xlabel('Time (days)')
            fig.suptitle("RA and DEC PM #{}".format(z))
            fig.tight_layout(pad=3.0)
            plt.tight_layout()
            pm_RA = (RA_values[1]-RA_values[0])/(time_values[1]-time_values[0])
            pm_DEC = (DEC_values[1]-DEC_values[0])/(time_values[1]-time_values[0])
            temp_pm = Table([])
            temp_pm['pm_RA'] = [pm_RA]
            temp_pm['pm_DEC'] = [pm_DEC]
            pm_table = vstack([pm_table, temp_pm])
        else:
            #For 3 or more measurements, still create a plot, but must measure proper motion using chi squared statistic
            fig = plt.figure(figsize=(15, 5))
            ax1 = fig.add_subplot(1,2,1)
            ax2 = fig.add_subplot(1,2,2)
            ax1.plot(time_values, RA_values, '-ok')
            ax2.plot(time_values, DEC_values, '-ok')
            ax1.set_ylabel('RA coords (deg)')
            ax1.set_xlabel('Time (days)')
            ax2.set_ylabel('DEC coords (deg)')
            ax2.set_xlabel('Time (days)')
            fig.suptitle("RA and DEC PM #{}".format(z))
            fig.tight_layout(pad=3.0)
            plt.tight_layout()
            
            RA_comb = combinations(RA_values, 2)
            time_comb = combinations(time_values, 2)
            DEC_comb = combinations(DEC_values, 2)
            combo_RA = []
            combo_time = []
            combo_DEC = []
            for i in list(RA_comb): 
                combo_RA.append(i) 
            for i in list(time_comb): 
                combo_time.append(i)
            for i in list(DEC_comb):
                combo_DEC.append(i)

            temp_pm = Table([])
            for y in range(len(list(combo_RA))):
                pmRA = (combo_RA[y][1]-combo_RA[y][0])/(combo_time[y][1]-combo_time[y][0]) #In degrees per day
                pmRA = pmRA*3600000 * 365.2422 #In milliarcseconds per year
                pmDEC = (combo_DEC[y][1]-combo_DEC[y][0])/(combo_time[y][1]-combo_time[y][0]) #In degrees per day
                pmDEC = pmDEC*3600000 * 365.2422 #In milliarcseconds per year
                temp_pm_table = Table([])
                temp_pm_table["pmRA#{} ".format(y)] = [pmRA]
                temp_pm_table["pmDEC#{} ".format(y)] = [pmDEC]
                #temp_pm["pmRA#{}".format(y)].unit = u.mas/u.year
                #temp_pm["pmDEC#{}".format(y)].unit = u.mas/u.year
                temp_pm = hstack([temp_pm, temp_pm_table])
            pm_table = vstack([pm_table, temp_pm])
    for x in range(len(list(info_table.colnames))):
        if 'pmDEC.' in info_table.colnames[x]:
            pm_table = hstack([info_table[info_table.colnames[x]], pm_table])
        if 'pmRA.' in info_table.colnames[x]:
            pm_table = hstack([info_table[info_table.colnames[x]], pm_table])
    
    return(pm_table)
    chi_tot = Table([])
    for z in range(len(list(pm_table[pm_table.colnames[0]]))):
        calcRA_values = []
        calcDEC_values = []
        measRA_values = []
        measDEC_values = []
        chiRA_values = []
        chiDEC_values = []
        
        for x in range(len(list(pm_table.colnames))):     
            if "RA#" in pm_table.colnames[x]:
                calcRA_values.append(pm_table[pm_table.colnames[x]][z])
            if "DEC#" in pm_table.colnames[x]:
                calcDEC_values.append(pm_table[pm_table.colnames[x]][z])
            if "RA." in pm_table.colnames[x]:
                measRA_values.append(pm_table[pm_table.colnames[x]][z])
            if "DEC." in pm_table.colnames[x]:
                measDEC_values.append(pm_table[pm_table.colnames[x]][z])    
            
            #All calculated proper motions along with the expected proper motion are now in there own lists.  Don't know what
            #to do if more than one measured value.  Will figure that out later
        for y in range(len(calcRA_values)):
            chi_sq_RA = ((calcRA_values[y] - measRA_values[0])**2) / measRA_values[0]
            chi_sq_DEC = ((calcDEC_values[y] - measDEC_values[0])**2) / measDEC_values[0]
            chiRA_values.append(chi_sq_RA)
            chiDEC_values.append(chi_sq_DEC)
        chiRA_tot = sum(chiRA_values)
        chiDEC_tot = sum(chiDEC_values)
        chi_temp = Table([])
        chi_temp["chiRA"] = [chiRA_tot]
        chi_temp["chiDEC"] = [chiDEC_tot]
        chi_tot = vstack([chi_tot, chi_temp])
        
    return(chi_tot)
    
    #To DO:
    #1. Understand how chi squared would apply to this function.  Maybe calculate proper motion of each possible combination,
    #   then calculate mean value of each proper motion based on normal distribution of error.
    
    #   Another promising chi squared is to calculate the proper motions of each possible combination, then subtract each one by
    #   the expected proper motion, most likely GAIA.  Square that outcome then divide by the expected proper motion.  Lastly,
    #   add up all the values to determine chi suqared
    #2. 
    #3. 
    
    
     
        

    #After error has been taken into account, two things need to happen.  First, all estimated proper motion values must be compared
    # to the measured proper motion from GAIA.  If the proper motion values do not fall within error margins, that source must be 
    # flagged.  If the proper motion values do match up, but the proper motion is moving below a certain speed (TBD), that source 
    # can be disregarded.
    
    
    #Logic Check:  If RA and DEC for one PANSTARRS source matches for two 2MASS sources, flag both sources
#    comp_table['Duplicate'] = 'NO'
#    PANSTARRS_coords = Table([comp_table['PANSTARRS-RAJ2000'], comp_table['PANSTARRS-DEJ2000']])
#    for x in range(len(list(PANSTARRS_coords))):
#       print(list(PANSTARRS_coords).count(PANSTARRS_coords[x]))
#        if list(PANSTARRS_coords).count(PANSTARRS_coords[x]) > 1:
#            print("Duplicate coordinates")
#            comp_table['Duplicate'][x] = 'YES'
#        else:
#            continue
    
    #If a source is flagged as having duplicate coordinates, run a new search through GAIA using same coordinates.
    #
            
    
#   return(PANSTARRS_coords)
    